In [7]:
### import necessary packages
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OrdinalEncoder

from sksurv.datasets import load_gbsg2
from sksurv.preprocessing import OneHotEncoder
from sksurv.ensemble import RandomSurvivalForest

import eli5
from eli5.sklearn import PermutationImportance

In [3]:
### setting random seed
random_state=20 # control reproducibility 
n_estimators=1000 # fit a Random Survival Forest comprising 1000 trees
min_samples_split=10 # The minimum number of samples required to split an internal node
min_samples_leaf=15 # The minimum number of samples required to be at a leaf node. 
max_features="sqrt" #The number of features to consider when looking for the best split,If “sqrt”, then max_features=sqrt(n_features)
n_jobs=-1 #The number of jobs to run in parallel for both fit and predict.-1 means using all processors.

In [4]:
### read data for survival prediction
X_train=pd.read_excel('./data/HCM_RSF_training_data_X.xlsx', sheet_name='Sheet1')
X_test=pd.read_excel('./data/HCM_RSF_testing_data_X.xlsx', sheet_name='Sheet1')
y_train=pd.read_excel('./data/HCM_RSF_training_data_Y.xlsx',sheet_name='Sheet1')
y_test=pd.read_excel('./data/HCM_RSF_testing_data_Y.xlsx',sheet_name='Sheet1')
### converted original data to standard data format
y_train.rename(columns={'Survival time after surgery (year)': 'ST','cardiovascular death':'State'},inplace=True)
y_train.loc[:,"State"]=y_train.loc[:,"State"].astype(bool)
data=np.zeros(190, dtype={'names':('cens','time'),'formats':('?', '<f8')})
data['cens']=y_train.loc[:,"State"]
data['time']=y_train.loc[:,"ST"]
y_train=data

y_test.rename(columns={'Survival time after surgery (year)': 'ST','cardiovascular death':'State'},inplace=True)
y_test.loc[:,"State"]=y_test.loc[:,"State"].astype(bool)
data=np.zeros(112, dtype={'names':('cens','time'),'formats':('?', '<f8')})
data['cens']=y_test.loc[:,"State"]
data['time']=y_test.loc[:,"ST"]
y_test=data
### pandas.core.frame.DataFrame for X; numpy.ndarray for y

In [5]:
# fit a Random Survival Forest model
rsf = RandomSurvivalForest(n_estimators=n_estimators,
                           min_samples_split=min_samples_split,
                           min_samples_leaf=min_samples_leaf,
                           max_features=max_features,
                           n_jobs=n_jobs,
                           random_state=random_state)
rsf.fit(X_train,y_train)

RandomSurvivalForest(max_features='sqrt', min_samples_leaf=15,
                     min_samples_split=10, n_estimators=1000, n_jobs=-1,
                     random_state=20)

In [6]:
### using all 922 metabolism as feature,and get a concordance index of 0.6827 in testing data
print('training data c-index: %f' % rsf.score(X_train,y_train))
print('testing data c-index: %f' % rsf.score(X_test,y_test))

training data c-index: 0.991721
testing data c-index: 0.682709


In [8]:
# Permutation-based Feature Importance
perm = PermutationImportance(rsf, n_iter=15, random_state=random_state)
perm.fit(X_test, y_test)
eli5.show_weights(perm, feature_names=X_train.columns.tolist())
#The result shows the importances of each features.for example,PC38:6p(16:0/22:6) If its relationship to survival time is removed (by random shuffling)
#the concordance index on the test data drops on average by 0.0150 points
### we select the top 12 VIMP(feature importance) of unique metabolics as feature to train a new model.As follows：
# ['Dimethylglycine',
#  'N-Acetyl-L-glutamine',
#  'gamma-Aminobutyric acid',
#  'XMP',
#  '18:0-Carnitine',
#  'GMP',
#  'UDP-galactose',
#  'PC38:6p(16:0/22:6)',
#  'PE32:0(16:0/16:0)',
#  'PS34:3(16:1/18:2)',
#  'PG38:6(18:2/20:4)',
#  'TAG52:2(C18:0)']

Weight,Feature
0.0150 ± 0.0188,PC38:6p(16:0/22:6)
0.0147 ± 0.0099,UDP-galactose
0.0128 ± 0.0205,gamma-Aminobutyric acid
0.0084 ± 0.0175,PG38:6(18:2/20:4)
0.0080 ± 0.0117,PC38:6p(18:2/20:4)
0.0077 ± 0.0068,GMP
0.0077 ± 0.0070,Dimethylglycine
0.0070 ± 0.0082,PE32:0(16:0/16:0)
0.0052 ± 0.0092,18:0-Carnitine
0.0052 ± 0.0158,PC38:4e(18:0/20:4)


In [9]:
### select the top 12 VIMP(feature importance) of unique metabolics as feature to train a new model
X_train=X_train.loc[:,['Dimethylglycine',
 'N-Acetyl-L-glutamine',
 'gamma-Aminobutyric acid',
 'XMP',
 '18:0-Carnitine',
 'GMP',
 'UDP-galactose',
 'PC38:6p(16:0/22:6)',
 'PE32:0(16:0/16:0)',
 'PS34:3(16:1/18:2)',
 'PG38:6(18:2/20:4)',
 'TAG52:2(C18:0)']]
X_test=X_test.loc[:,['Dimethylglycine',
 'N-Acetyl-L-glutamine',
 'gamma-Aminobutyric acid',
 'XMP',
 '18:0-Carnitine',
 'GMP',
 'UDP-galactose',
 'PC38:6p(16:0/22:6)',
 'PE32:0(16:0/16:0)',
 'PS34:3(16:1/18:2)',
 'PG38:6(18:2/20:4)',
 'TAG52:2(C18:0)']]

In [10]:
rsf = RandomSurvivalForest(n_estimators=1000,
                           min_samples_split=10,
                           min_samples_leaf=15,
                           max_features="sqrt",
                           n_jobs=-1,
                           random_state=random_state)
rsf.fit(X_train,y_train)

RandomSurvivalForest(max_features='sqrt', min_samples_leaf=15,
                     min_samples_split=10, n_estimators=1000, n_jobs=-1,
                     random_state=20)

In [11]:
### using the top 12 VIMP of unique metabolics as feature,and get a concordance index of 0.916221 in testing data
print('training data c-index: %f' % rsf.score(X_train,y_train))
print('testing data c-index: %f' % rsf.score(X_test,y_test))

training data c-index: 0.982259
testing data c-index: 0.916221


In [ ]:
# Permutation-based Feature Importance
perm = PermutationImportance(rsf, n_iter=15, random_state=random_state)
perm.fit(X_test, y_test)
eli5.show_weights(perm, feature_names=X_train.columns.tolist())

In [12]:
### calculate the 95% CI(confidence interval) of C-index
n_bootstraps=1000
rng_seed=random_state
def CI(X_test,y_test):
    bootstrapped_scores = []
    C_index=rsf.score(X_test,y_test)
    print("C-index:",rsf.score(X_test,y_test))
    rng = np.random.RandomState(rng_seed)
    for i in range(n_bootstraps):
        # bootstrap by sampling with replacement on the prediction indices
        indices = rng.randint(0, len(y_test), len(y_test))
        if len(np.unique(y_test[indices]['cens']))<2:
            continue
        score=rsf.score(X_test.loc[indices],y_test[indices])
        bootstrapped_scores.append(score)
        #print("Bootstrap #{} c-index: {:0.3f}".format(i + 1, score))
    sorted_scores = np.array(bootstrapped_scores)
    sorted_scores.sort()
    # Computing the lower and upper bound of the 90% confidence interval
    # You can change the bounds percentiles to 0.025 and 0.975 to get
    # a 95% confidence interval instead.
    confidence_lower = sorted_scores[int(0.025 * len(sorted_scores))]
    confidence_upper = sorted_scores[int(0.975 * len(sorted_scores))]
    print("Confidence interval for the score: [{:0.3f} - {:0.3}]".format(
        confidence_lower, confidence_upper))
    return C_index,confidence_lower,confidence_upper
C_index,confidence_lower,confidence_upper=CI(X_test,y_test)
print('RSF:C-index={:0.3f}\n95%CI: {:0.3f} - {:0.3f}'.format(C_index,confidence_lower,confidence_upper))

C-index: 0.9162210338680927
Confidence interval for the score: [0.814 - 0.978]
RSF:C-index=0.916
95%CI: 0.814 - 0.978
